# Elliptic Filter Stage Coefficient Formulas

In [1]:
from sympy import symbols, collect, cancel, simplify
from sympy import fraction, degree


In [2]:
s, z, cden1, cden0, cnum0, K, Q = symbols('s z cden1 cden0 cnum0 K Q')


Laplace domain low pass prototype filter stage for the elliptic filter.

In [3]:
h_s_lp = (s**2 + cnum0)/(s**2 + cden1*s + cden0)


To find the equivalent digital filter the bilinear transform with frequency warping is used. This assumes that the cutoff frequency of the prototype filter is 1 rad/s.

K = 1/tan(w/2)

w = 2\*π\*f_c/f_s

f_c = digital cutoff frequency (or center frequency)

f_s = sampling frequency

π = pi


In [4]:
sz = K*(z-1)/(z+1)


The bilinear transform is substituted for s in the Laplace transform to obtain the desired digital filter and variations.

For the band pass and band stop filters Q determines the bandwidth according to the following formula:

Q = f_c/(f_2 - f_1)

f_c = sqrt(f_2\*f_1)

f_1 = lower cutoff frequency

f_2 = upper cutoff frequency

For example f_c and Q can be calculated for an analog telephone line simulation filter.

In [5]:
f_1_tp = 300
f_2_tp = 3400
f_c_tp = (f_1_tp*f_2_tp)**0.5
Q_tp = f_c_tp/(f_2_tp-f_1_tp)
print(f"f_c_tp: {f_c_tp:.0f} Q_tp: {Q_tp:.3f}")

f_c_tp: 1010 Q_tp: 0.326


In [6]:
h_z_lp = h_s_lp.subs(s, sz)
h_z_hp = h_s_lp.subs(s, 1/sz)
h_z_bp = h_s_lp.subs(s, Q*(sz+1/sz))
h_z_bs = h_s_lp.subs(s, 1/(Q*(sz+1/sz)))


With the digital filters it is possible to print the coefficient equations. Note that the coefficients are reversed. This is because the given functions are non-causal and the function for the actual implementation needs to be causal.

For examples of using these equations have a look at the following source code files:
* [elliptical_lp.c](elliptical_lp.c) for the low pass filter
* [elliptical_hp.c](elliptical_hp.c) for the high pass filter
* [elliptical_bp.c](elliptical_bp.c) for the band pass filter
* [elliptical_bs.c](elliptical_bs.c) for the band stop filter

In [7]:
def print_poly_coefficients(hz, z, v, Q):
    d = degree(hz, gen=z)
    for i in range(d+1):
        print(v,i," = ",collect(hz.coeff(z,d-i),Q),sep='')
    print()

def print_coefficients(hz, z, Q):
    hz = collect(cancel(simplify(hz)), z)
    hz_num, hz_den = fraction(hz)
    print("Numerator coefficients (causal)")
    print_poly_coefficients(hz_num, z, 'b', Q)
    print("Denominator coefficients (causal)")
    print_poly_coefficients(hz_den, z, 'a', Q)


In [8]:
hzs = [
    (h_z_lp,"low pass"),
    (h_z_hp,"high pass"),
    (h_z_bp,"band pass"),
    (h_z_bs,"band stop") ]

for hz, s in hzs:
    print("*"*80)
    print(s)
    print_coefficients(hz, z, Q)


********************************************************************************
low pass
Numerator coefficients (causal)
b0 = K**2 + cnum0
b1 = -2*K**2 + 2*cnum0
b2 = K**2 + cnum0

Denominator coefficients (causal)
a0 = K**2 + K*cden1 + cden0
a1 = -2*K**2 + 2*cden0
a2 = K**2 - K*cden1 + cden0

********************************************************************************
high pass
Numerator coefficients (causal)
b0 = K**2*cnum0 + 1
b1 = -2*K**2*cnum0 + 2
b2 = K**2*cnum0 + 1

Denominator coefficients (causal)
a0 = K**2*cden0 + K*cden1 + 1
a1 = -2*K**2*cden0 + 2
a2 = K**2*cden0 - K*cden1 + 1

********************************************************************************
band pass
Numerator coefficients (causal)
b0 = K**2*cnum0 + Q**2*(K**4 + 2*K**2 + 1)
b1 = Q**2*(4 - 4*K**4)
b2 = -2*K**2*cnum0 + Q**2*(6*K**4 - 4*K**2 + 6)
b3 = Q**2*(4 - 4*K**4)
b4 = K**2*cnum0 + Q**2*(K**4 + 2*K**2 + 1)

Denominator coefficients (causal)
a0 = K**2*cden0 + Q**2*(K**4 + 2*K**2 + 1) + Q*(K**3*cden1 + 